In [ ]:
# Purpose: 		automatic marking of 11+ multipe choice answer sheets.
#
# Overview:     1. user inputs variables such as paper name, folder path of scanned answer sheets and number of students.
#               2. code loads in pre-saved variables relating to the specific mock paper.
#               3. loaded variables are used to identify which answer boxes have been marked on the completed answer sheets.
#               4. student's answers are compared to answer key and a score is calculated.
#               5. code loops over all students and returns an array of marks
#
# Input(s):		paper = name of mock paper (string)
#               folder = folder path of scanned answer sheets (string)
#               nS = number of students (scalar)
#
# Output(s): 	paper_marks.csv = a .csv file of the students' marks (.csv file)

In [ ]:
# inputs

paper = 'Mock2E'
folder = 'C:/Users/jimmy/OneDrive/Tutor World OneDrive/11+/Mocks - TW/2022/Mock 2/AutoMarker'
nS = 1

In [ ]:
# imports

import numpy as np
import cv2
import pickle

In [ ]:
# main code
# work out which boxes have been marked and compare to markscheme

# load in dictionary of variables and unpack
var_dict = pickle.load(open(paper+".p", "rb" ) )
nQ = var_dict["nQ"]
nOpts = var_dict["nOpts"]
coords = var_dict["coords"]
MS = var_dict["MS"]

# initialise array for marks
marks = np.zeros((nS,nQ),np.int8)

# function to store the coordinates
# of the points clicked on the image
def click_event(event, x, y, flags, params):

	global box_1_coords

	# on left mouse click, store the coordinates and iterate i and k

	if event == cv2.EVENT_LBUTTONDOWN:

		box_1_coords.append((x,y))

		# close the window
		cv2.destroyAllWindows()

for s in range(nS):

	# read in the image
	img = cv2.imread(folder+"/"+paper+"_Page_"+str(s+1)+".png", 0)

	# resize image
	scale_percent = 15
	width = int(img.shape[1] * scale_percent / 100)
	height = int(img.shape[0] * scale_percent / 100)
	dim = (width, height)
	img = cv2.resize(img, dim, interpolation = cv2.INTER_AREA)

	# initialise variables
	box_1_coords = []
	AS = [[] for i in range(nQ)]
	AS_marks = [[] for i in range(nQ)]

	# display the image
	cv2.imshow('image', img)

	# set mouse handler for the image
	# and call the click_event() function
	cv2.setMouseCallback('image', click_event)

	# wait for a key to be pressed to move on
	cv2.waitKey(0)

	# calculate displacement of answer sheet from original
	x_disp = box_1_coords[0][0] - coords[0][0][0]
	y_disp = box_1_coords[0][1] - coords[0][0][1]
	disp = [(x_disp,y_disp)]

	# work out which boxes have been marked and compare to markscheme
	for i in range(nQ):
		for j in range(nOpts[i]):

			# get mean pixel intensity in small rectangle around coordinate
			AS_marks[i].append(np.mean(img[coords[i][j][1]-4+disp[0][1]:coords[i][j][1]+4+disp[0][1],coords[i][j][0]-12+disp[0][0]:coords[i][j][0]+12+disp[0][0]]))

			# identify box as having been marked if intensity if below threshold
			if  AS_marks[i][j] < 190:
				AS[i].append(True)
			else:
				AS[i].append(False)

		# compare marked answer boxes to mark scheme	
		if MS[i] == AS[i]:
			marks[s,i] = 1

marks

In [ ]:
# output marks to .csv

np.savetxt(folder+"/"+paper+"_marks.csv", marks, delimiter=",", fmt = '%i')

In [ ]:
# tests

AS_marks